In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import sklearn 
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from scipy import stats  # Import stats module from SciPy
from matplotlib import rcParams
rcParams['font.family'] = 'Times New Roman'
rcParams['font.size'] = 14

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import StackingRegressor

# Load sample dataset (you can replace this with your actual dataset)
df = pd.read_csv('sample_data.csv')

# Assuming 'X' contains the features and 'y' contains the target variable
X = df.drop(columns=['TargetColumn'])
y = df['TargetColumn']

# Eliminate outliers using Z-score method
z_scores = np.abs(stats.zscore(df))
threshold = 3
df = df[(z_scores < threshold).all(axis=1)]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

# List of regression algorithms
regression_algorithms = [
    RandomForestRegressor(n_estimators=100),
    GradientBoostingRegressor(n_estimators=100),
    AdaBoostRegressor(n_estimators=100),
    XGBRegressor(n_estimators=100),
    CatBoostRegressor(silent=True, n_estimators=100)
]

# Dictionary to store the evaluation results
evaluation_results = {
    "Algorithm": [],
    "MAE": [],
    "MSE": [],
    "R2": []
}

# Create subplots for each algorithm's evaluation
num_algorithms = len(regression_algorithms)
num_cols = 2
num_rows = (num_algorithms + num_cols - 1) // num_cols

fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 10))
axes = axes.flatten()

# Initialize list to store base model predictions
base_model_predictions = []

# Evaluate each algorithm and plot subplots
for i, algorithm in enumerate(regression_algorithms):
    algorithm_name = algorithm.__class__.__name__
    algorithm.fit(X_train, y_train)
    y_pred = algorithm.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    evaluation_results["Algorithm"].append(algorithm_name)
    evaluation_results["MAE"].append(mae)
    evaluation_results["MSE"].append(mse)
    evaluation_results["R2"].append(r2)

    # Store base model prediction for stacking
    base_model_predictions.append(y_pred)

    # Plot ideal line for each algorithm in its corresponding subplot
    ax = axes[i]
    ax.scatter(y_test, y_pred, alpha=0.8, facecolors='none', edgecolors='b')
    ax.plot(y_test, y_test, color='red', linestyle='--')
    ax.set_xlabel('Actual Target')
    ax.set_ylabel('Predicted Target')
    ax.set_title(f"{algorithm_name}", fontsize=16)

    # Set background color to white
    ax.set_facecolor('white')

    # Display R2 score on the plot
    ax.text(0.05, 0.95, f'R2: {r2:.2f}', transform=ax.transAxes, fontsize=16, verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

# Create stacking model with Linear Regression as meta-learner
stacking_model = StackingRegressor(estimators=regression_algorithms, final_estimator=LinearRegression())

# Fit stacking model on base model predictions
stacking_model.fit(X_train, y_train)

# Predict using stacking model
stacked_test_predictions = stacking_model.predict(X_test)

# Calculate R2 score for stacking model
stacking_r2 = r2_score(y_test, stacked_test_predictions)

# Store stacking model results in evaluation results dictionary
evaluation_results["Algorithm"].append("Stacking")
evaluation_results["MAE"].append(mean_absolute_error(y_test, stacked_test_predictions))
evaluation_results["MSE"].append(mean_squared_error(y_test, stacked_test_predictions))
evaluation_results["R2"].append(stacking_r2)

# Plot ideal line for stacking in its corresponding subplot
ax = axes[num_algorithms]
ax.scatter(y_test, stacked_test_predictions, alpha=0.8, facecolors='none', edgecolors='b')
ax.plot(y_test, y_test, color='red', linestyle='--')
ax.set_xlabel('Actual Target')
ax.set_ylabel('Predicted Target')
ax.set_title("Stacking", fontsize=16)
ax.set_facecolor('white')

# Display R2 score on the plot
ax.text(0.05, 0.95, f'R2: {stacking_r2:.2f}', transform=ax.transAxes, fontsize=16, verticalalignment='top',
        bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

# Adjust layout and show plots
plt.tight_layout()
plt.show()

# Convert the evaluation results to a DataFrame
evaluation_df = pd.DataFrame(evaluation_results)

# Display the evaluation results
print(evaluation_df)

print(X_test)
